# Catboost

Tien-Thinh

In [1]:
import numpy as np
import pandas as pd
from scipy.stats import spearmanr
from sklearn.linear_model import LinearRegression
import matplotlib.pyplot as plt

from catboost import CatBoostRegressor
from sklearn.metrics import mean_absolute_error 
from sklearn.model_selection import train_test_split 

In [2]:
X_train = pd.read_csv('X_train.csv')
Y_train = pd.read_csv('Y_train.csv')
X_test = pd.read_csv('X_test.csv')

In [3]:
X_train

,ID,DAY_ID,COUNTRY,DE_CONSUMPTION,FR_CONSUMPTION,DE_FR_EXCHANGE,FR_DE_EXCHANGE,DE_NET_EXPORT,FR_NET_EXPORT,DE_NET_IMPORT,...,FR_RESIDUAL_LOAD,DE_RAIN,FR_RAIN,DE_WIND,FR_WIND,DE_TEMP,FR_TEMP,GAS_RET,COAL_RET,CARBON_RET
0,1054,206,FR,0.210099,-0.427458,-0.606523,0.606523,NaN,0.692860,NaN,...,-0.444661,-0.172680,-0.556356,-0.790823,-0.283160,-1.069070,-0.063404,0.339041,0.124552,-0.002445
1,2049,501,FR,-0.022399,-1.003452,-0.022063,0.022063,-0.573520,-1.130838,0.573520,...,-1.183194,-1.240300,-0.770457,1.522331,0.828412,0.437419,1.831241,-0.659091,0.047114,-0.490365
2,1924,687,FR,1.395035,1.978665,1.021305,-1.021305,-0.622021,-1.682587,0.622021,...,1.947273,-0.480700,-0.313338,0.431134,0.487608,0.684884,0.114836,0.535974,0.743338,0.204952
3,297,720,DE,-0.983324,-0.849198,-0.839586,0.839586,-0.270870,0.563230,0.270870,...,-0.976974,-1.114838,-0.507570,-0.499409,-0.236249,0.350938,-0.417514,0.911652,-0.296168,1.073948
4,1101,818,FR,0.143807,-0.617038,-0.924990,0.924990,NaN,0.990324,NaN,...,-0.526267,-0.541465,-0.424550,-1.088158,-1.011560,0.614338,0.729495,0.245109,1.526606,2.614378
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1489,459,809,DE,1.529204,1.106682,-1.855327,1.855327,-0.218658,1.450426,0.218658,...,0.509514,NaN,NaN,NaN,NaN,NaN,NaN,0.876984,0.819520,1.320373
1490,1674,887,FR,1.618582,1.752840,0.611392,-0.611392,0.449153,-0.152146,-0.449153,...,1.666252,NaN,NaN,NaN,NaN,NaN,NaN,0.932633,-0.085690,0.356356
1491,748,1083,DE,0.856399,0.489199,-0.255778,0.255778,-1.531544,-0.829568,1.531544,...,0.358120,0.207905,0.404763,-0.594595,0.894011,0.256338,0.402316,-1.112899,-0.237835,0.067152
1492,1454,1133,FR,0.560689,-0.343777,-0.830239,0.830239,-0.304856,1.210230,0.304856,...,-0.184862,-0.682815,-0.390304,-0.972088,-1.501930,1.215528,1.338708,0.962812,-5.392852,-0.843812


In [47]:
len(X_train.DAY_ID.unique())*2

1702

# Test NAN

In [7]:
sum(np.isnan(Y_train).ID)

0

In [12]:
sum(Y_train.ID == X_train.ID)

1494

In [17]:
X_train.isna().describe()

,ID,DAY_ID,COUNTRY,DE_CONSUMPTION,FR_CONSUMPTION,DE_FR_EXCHANGE,FR_DE_EXCHANGE,DE_NET_EXPORT,FR_NET_EXPORT,DE_NET_IMPORT,...,FR_RESIDUAL_LOAD,DE_RAIN,FR_RAIN,DE_WIND,FR_WIND,DE_TEMP,FR_TEMP,GAS_RET,COAL_RET,CARBON_RET
count,1494,1494,1494,1494,1494,1494,1494,1494,1494,1494,...,1494,1494,1494,1494,1494,1494,1494,1494,1494,1494
unique,1,1,1,1,1,2,2,2,2,2,...,1,2,2,2,2,2,2,1,1,1
top,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
freq,1494,1494,1494,1494,1494,1469,1469,1370,1424,1370,...,1494,1400,1400,1400,1400,1400,1400,1494,1494,1494


In [18]:
X_test.isna().describe()

,ID,DAY_ID,COUNTRY,DE_CONSUMPTION,FR_CONSUMPTION,DE_FR_EXCHANGE,FR_DE_EXCHANGE,DE_NET_EXPORT,FR_NET_EXPORT,DE_NET_IMPORT,...,FR_RESIDUAL_LOAD,DE_RAIN,FR_RAIN,DE_WIND,FR_WIND,DE_TEMP,FR_TEMP,GAS_RET,COAL_RET,CARBON_RET
count,654,654,654,654,654,654,654,654,654,654,...,654,654,654,654,654,654,654,654,654,654
unique,1,1,1,1,1,2,2,2,2,2,...,1,2,2,2,2,2,2,1,1,1
top,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
freq,654,654,654,654,654,645,645,607,630,607,...,654,614,614,614,614,614,614,654,654,654


In [25]:
Y_train

,ID,TARGET
0,1054,0.028313
1,2049,-0.112516
2,1924,-0.180840
3,297,-0.260356
4,1101,-0.071733
...,...,...
1489,459,-0.172597
1490,1674,-0.063546
1491,748,0.151797
1492,1454,-0.640917


# trainning data

In [19]:
X_train.columns

Index(['ID', 'DAY_ID', 'COUNTRY', 'DE_CONSUMPTION', 'FR_CONSUMPTION',
       'DE_FR_EXCHANGE', 'FR_DE_EXCHANGE', 'DE_NET_EXPORT', 'FR_NET_EXPORT',
       'DE_NET_IMPORT', 'FR_NET_IMPORT', 'DE_GAS', 'FR_GAS', 'DE_COAL',
       'FR_COAL', 'DE_HYDRO', 'FR_HYDRO', 'DE_NUCLEAR', 'FR_NUCLEAR',
       'DE_SOLAR', 'FR_SOLAR', 'DE_WINDPOW', 'FR_WINDPOW', 'DE_LIGNITE',
       'DE_RESIDUAL_LOAD', 'FR_RESIDUAL_LOAD', 'DE_RAIN', 'FR_RAIN', 'DE_WIND',
       'FR_WIND', 'DE_TEMP', 'FR_TEMP', 'GAS_RET', 'COAL_RET', 'CARBON_RET'],
      dtype='object')

In [20]:
X_train_clean = X_train.drop(['ID'], axis=1)
Y_train_clean = Y_train['TARGET']

In [22]:
X_train_split, X_test_split, y_train_split, y_test_split = train_test_split(
    X_train_clean, Y_train_clean, test_size=0.03
)

In [23]:
X_train_split

,DAY_ID,COUNTRY,DE_CONSUMPTION,FR_CONSUMPTION,DE_FR_EXCHANGE,FR_DE_EXCHANGE,DE_NET_EXPORT,FR_NET_EXPORT,DE_NET_IMPORT,FR_NET_IMPORT,...,FR_RESIDUAL_LOAD,DE_RAIN,FR_RAIN,DE_WIND,FR_WIND,DE_TEMP,FR_TEMP,GAS_RET,COAL_RET,CARBON_RET
705,225,DE,0.052397,-0.752295,-0.643194,0.643194,-1.117993,-0.850813,1.117993,0.850813,...,-0.923226,0.421359,1.089102,0.268746,0.928706,0.478052,0.897946,0.676415,-0.420079,-1.157321
1247,503,FR,-0.501270,-0.796449,-0.480015,0.480015,-0.792889,-0.723719,0.792889,0.723719,...,-0.885554,0.574685,-0.486974,-0.409457,-0.262854,-0.326737,-1.061768,-0.645343,0.464638,0.263928
1009,97,FR,-0.028612,-0.805939,-1.130925,1.130925,-1.737154,1.303680,1.737154,-1.303680,...,-0.746805,-1.065734,0.269952,0.179885,0.453358,-0.073011,-1.686025,1.288241,-0.033589,0.141656
573,88,DE,0.132381,-0.719676,-0.503860,0.503860,-0.601611,-0.702669,0.601611,0.702669,...,-0.900449,-0.305152,0.734147,-0.647389,-0.356483,1.162059,0.591288,0.877416,-1.530145,0.456988
1272,354,FR,1.330637,2.313961,NaN,NaN,NaN,NaN,NaN,NaN,...,2.372387,-0.389086,1.597882,-1.052459,-0.642847,-1.419704,-1.328871,-1.658968,-1.171063,0.512507
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1474,1017,FR,-0.194949,-0.693055,-1.007199,1.007199,-0.661682,0.454472,0.661682,-0.454472,...,-0.730173,-0.447087,-0.172117,-0.712770,-0.668385,-1.160605,-1.627199,0.234032,-0.823336,0.217175
1405,792,FR,1.017328,0.731576,0.204565,-0.204565,-0.493411,-0.529358,0.493411,0.529358,...,0.675956,0.156814,-0.816840,0.350995,0.308940,2.011596,0.304000,2.410938,-0.843294,1.774454
1449,460,DE,0.797466,0.528192,0.893058,-0.893058,-0.185466,-0.871071,0.185466,0.871071,...,0.518742,-0.536307,-0.634404,0.539445,0.151308,-0.585758,-1.152746,-0.716368,-0.274941,0.198548
986,1212,DE,-0.046100,-0.520506,-0.270515,0.270515,-0.137917,-0.789824,0.137917,0.789824,...,-0.589767,-0.039379,-0.237434,1.358211,1.123953,-0.707839,-0.308232,0.017778,0.072168,-0.160792


In [39]:
model = CatBoostRegressor(iterations=10, eval_metric='spearman')

In [50]:
model = CatBoostRegressor(iterations=10, loss_function='MAE', eval_metric='spearman')

In [51]:
model.fit(
    X_train_split, y_train_split, 
    cat_features=["COUNTRY", "DAY_ID"], 
    eval_set=(X_test_split, y_test_split), 
    use_best_model=True
)

CatBoostError: /src/catboost/catboost/python-package/catboost/helpers.cpp:49: Traceback (most recent call last):
  File "_catboost.pyx", line 1163, in _catboost._MetricIsMaxOptimal
AttributeError: 'function' object has no attribute 'is_max_optimal'
